# Worksheet 1: Types

1. What happens if you try to instantiate a cme257int with a float?  Why did this happen?

2. Create a type called "cme257OrderedPair" with the following properties:
    1. It is a child of cme257abstract
    2. It has two fields: "a" and "b"
    3. The fields "a" and "b" are the same parameterized type
    
3. How do you create a Complex number in Julia?  Investigate the resulting type:
    1. Is it a primitive type?  If not, what fields does it have?
    2. What else do you notice?  Is it immutable?  Is it parameterized?

In [1]:
abstract type cme257abstract end

In [2]:
# an abstract type for our real number types
abstract type cme257real <: cme257abstract end

struct cme257int <: cme257real # <: denotes "child of"
    x::Int64 # :: tells us exactly what type x should be
end
struct cme257float <: cme257real
    x::Float64
end

In [3]:
#a0 = cme257int(5.0)
a1 = cme257int(5.5)

LoadError: [91mInexactError()[39m

In [4]:
struct cme257OrderedPair{T} <: cme257abstract
    a::T
    b::T
end

In [5]:
op1 = cme257OrderedPair(3,4)
op1.a

3

In [6]:
z = 5.0 + 1.0im

5.0 + 1.0im

In [7]:
typeof(z)

Complex{Float64}

In [8]:
# fields of complex numbers
@show z.re
@show z.im
z.re = 3.0 # immutable
;

z.re = 5.0
z.im = 1.0


LoadError: [91mtype Complex is immutable[39m

# Worksheet 2: Functions

1. Overload the addition and multiplication operators to do element-wise addition and multiplication on cme257OrderedPair.

2. Overload addition and multiplication to work with cme257ff{N,T} for N any positive integer (don't worry about checking positivity).

(a +/* b).val = a.val +/* b.val mod N

Generally, this gives this type the structure of a ring.  If N is prime, the type has the structure of a field.

3. Modify ```yell_my_type``` to yell all types correctly.

In [9]:
import Base.+, Base.*

x = cme257OrderedPair(2, 3)
y = cme257OrderedPair(3, 4)

function +{T}(x::cme257OrderedPair{T}, y::cme257OrderedPair{T})
    return cme257OrderedPair(x.a + y.a, x.b + y.b) 
end

@show x + y

function *{T}(x::cme257OrderedPair{T}, y::cme257OrderedPair{T})
    return cme257OrderedPair(x.a * y.a, x.b * y.b)
end

@show x * y
;

x + y = cme257OrderedPair{Int64}(5, 7)
x * y = cme257OrderedPair{Int64}(6, 12)


In [10]:
# copied from lecture notebook

# we'll use this for finite fields with characteristic N
struct cme257ff{N, T<:Integer} #<: cme257abstract
    val::T
    # override the default constructor to store things mod N
    cme257ff{N,T}(val::T) where {N,T<:Integer} = new(mod(val,N))
end

# You'll also see the N convention in Julia with types like Array{T,N}, where T is a type and N is a number

# This will create constructors where the type of the value is inferred
cme257ff{N}(val::T) where {N, T} = cme257ff{N,T}(val)
# this will create constructors where signed integers are converted to unsigned integers
cme257ff{N,T}(val::T) where {N, T<:Signed} = cme257ff{N}(Unsigned(val))
# Note that the above are examples of creating functions

Base.show{N,T}(io::IO, x::cme257ff{N,T}) = print(io, x.val, " mod ", N, " (", T, ")")

In [11]:
function *{N,T}(a::cme257ff{N,T}, b::cme257ff{N,T})
    cme257ff{N,T}(T(mod(a.val * b.val, N)))
end

function +{N,T}(a::cme257ff{N,T}, b::cme257ff{N,T})
    cme257ff{N,T}(T(mod(a.val + b.val,N)))
end

x = cme257ff{5}(2)
y = cme257ff{5}(4)
@show x*y # 2 * 4 = 8 = 3 mod 5
@show x+y # 2 + 4 = 6 = 1 mod 5
;

x * y = 3 mod 5 (UInt64)
x + y = 1 mod 5 (UInt64)


In [12]:
# note that with our cme257ff type, we can't add or multiply elements of different rings
x = cme257ff{5}(2)
y = cme257ff{7}(4)
@show x+y
;

LoadError: [91mMethodError: no method matching +(::cme257ff{5,UInt64}, ::cme257ff{7,UInt64})[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  +(::cme257ff{N,T}, [91m::cme257ff{N,T}[39m) where {N, T} at In[11]:6[39m

In [13]:
# we also need the same underlying type
x = cme257ff{5}(UInt16(2))
y = cme257ff{5}(UInt32(4))
@show x*y
;

LoadError: [91mMethodError: no method matching *(::cme257ff{5,UInt16}, ::cme257ff{5,UInt32})[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  *(::cme257ff{N,T}, [91m::cme257ff{N,T}[39m) where {N, T} at In[11]:2[39m

In [14]:
function yell_my_type{T}(x::T)
   println(uppercase("MY TYPE IS $(T)"))
end

yell_my_type(5//7)

MY TYPE IS RATIONAL{INT64}
